<a href="https://colab.research.google.com/github/CathalD/NorthStarProject_CoastalBlueCarbonMMRV/blob/main/CoastalBlueCarbon_LargeScaleCovariateExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='north-star-project-470316')

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
# 2. LOAD AND FILTER DATA
FILENAME = 'global_cores_harmonized_VM0033.csv'
try:
    df = pd.read_csv(FILENAME)
except FileNotFoundError:
    print("❌ ERROR: File not found. Please upload the CSV.")
    raise

# Filter for specific ecosystems and West Coast latitudes (excluding Alaska)
target_ecosystems = ['EM', 'SG', 'FL']
MIN_LAT, MAX_LAT = 46.5, 55.5

df_subset = df[
    (df['ecosystem'].isin(target_ecosystems)) &
    (df['latitude'] >= MIN_LAT) &
    (df['latitude'] <= MAX_LAT)
].copy()

# Prepare Features
unique_locs = df_subset[['core_id', 'longitude', 'latitude']].drop_duplicates(subset='core_id').dropna()
feature_list = []
for _, row in unique_locs.iterrows():
    geom = ee.Geometry.Point([row['longitude'], row['latitude']])
    feature_list.append(ee.Feature(geom, {'core_id': str(row['core_id'])}))

print(f"✓ Processing {len(feature_list)} unique points (Lat {MIN_LAT}-{MAX_LAT})")

✓ Processing 182 unique points (Lat 46.5-55.5)


In [ ]:
# ============================================================================
# 3. DEFINE LAYERS (BRIDGE 21)
# ============================================================================

# --- A. TOPOGRAPHY ---
dem = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
elevation_m = dem.rename("elevation_m")
slope = ee.Terrain.slope(dem).rename("slope")

# Proxy for MHW (Mean High Water).
# Global approximation: Elevation - 0.5m (Rough estimate of MHW above MSL)
# In local script, use your precise local datum.
elevationRelMHW = dem.subtract(0.5).rename("elevationRelMHW")

stack_topo = elevation_m.addBands(slope).addBands(elevationRelMHW)

# --- B. SENTINEL-1 (SAR) ---
s1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
      .filterDate('2023-01-01', '2023-12-31')
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .filter(ee.Filter.eq('instrumentMode', 'IW')))

# Mean calculation (User requested Mean for SAR)
s1_mean = s1.mean()

vv = s1_mean.select('VV').rename('VV_mean')
vh = s1_mean.select('VH').rename('VH_mean')
ratio = s1_mean.select('VV').subtract(s1_mean.select('VH')).rename('VVVH_ratio')

stack_s1 = vv.addBands(vh).addBands(ratio)

# --- C. SENTINEL-2 (OPTICAL + TIDAL MASKING) ---

def process_s2_image(image):
    # 1. Cloud Masking
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1<<10).eq(0).bitwiseAnd(1<<11).eq(0)

    # 2. Tidal/Water Masking (NDWI-based)
    # If NDWI > 0.1, we assume it's inundated/water. We mask it out.
    # This ensures the Median is calculated only from EXPOSED soil/veg.
    ndwi_check = image.normalizedDifference(['B3', 'B8'])
    tide_mask = ndwi_check.lt(0.1) # Keep only if NDWI < 0.1 (Land/Exposed)

    # Combine masks
    final_mask = cloud_mask.And(tide_mask)

    return image.updateMask(final_mask).divide(10000)

s2_coll = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
           .filterDate('2023-01-01', '2023-12-31')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
           .map(process_s2_image))

# Calculate Median of EXPOSED pixels
s2 = s2_coll.median()

# Bands
B = s2.select('B2').rename('B')
G = s2.select('B3').rename('G')
R = s2.select('B4').rename('R')
NIR = s2.select('B8').rename('NIR')
SWIR1 = s2.select('B11').rename('SWIR1')
SWIR2 = s2.select('B12').rename('SWIR2')

# Indices
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI_median')
ndbi = s2.normalizedDifference(['B11', 'B8']).rename('NDBI_median') # SWIR1/NIR
evi_expr = '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))'
evi = s2.expression(evi_expr, {'NIR': NIR, 'RED': R, 'BLUE': B}).rename('EVI_median')
savi = s2.expression('((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {'NIR': NIR, 'RED': R}).rename('SAVI_median')
lswi = s2.normalizedDifference(['B8', 'B11']).rename('LSWI_median') # NIR/SWIR1
mndwi = s2.normalizedDifference(['B3', 'B11']).rename('mNDWI_median') # Green/SWIR1

# Tasseled Cap (Sentinel-2 Coefficients)
brightness = s2.expression(
    '0.3037*B2 + 0.2793*B3 + 0.4743*B4 + 0.5585*B8 + 0.5082*B11 + 0.1863*B12',
    {'B2':B, 'B3':G, 'B4':R, 'B8':NIR, 'B11':SWIR1, 'B12':SWIR2}
).rename('brightness')

greenness = s2.expression(
    '-0.2848*B2 - 0.2435*B3 - 0.5436*B4 + 0.7243*B8 + 0.0840*B11 - 0.1800*B12',
    {'B2':B, 'B3':G, 'B4':R, 'B8':NIR, 'B11':SWIR1, 'B12':SWIR2}
).rename('greenness')

wetness = s2.expression(
    '0.1509*B2 + 0.1973*B3 + 0.3279*B4 + 0.3406*B8 - 0.7112*B11 - 0.4572*B12',
    {'B2':B, 'B3':G, 'B4':R, 'B8':NIR, 'B11':SWIR1, 'B12':SWIR2}
).rename('wetness')

stack_s2 = (B.addBands(G).addBands(R).addBands(NIR).addBands(SWIR1).addBands(SWIR2)
            .addBands(ndvi).addBands(ndbi).addBands(evi).addBands(savi)
            .addBands(lswi).addBands(mndwi)
            .addBands(brightness).addBands(greenness).addBands(wetness))

In [ ]:
# ============================================================================
# 4. BATCH EXTRACTION (SAFE MODE)
# ============================================================================
def extract_batch(image, features, name, batch_size=50):
    results = []
    print(f"--- Extracting {name} ---")
    for i in range(0, len(features), batch_size):
        batch = features[i : i + batch_size]
        fc = ee.FeatureCollection(batch)
        try:
            # Spatially constrained extraction
            if "Sentinel" in name:
                res = image.clip(fc.geometry().buffer(100)).reduceRegions(collection=fc, reducer=ee.Reducer.first(), scale=30)
            else:
                res = image.reduceRegions(collection=fc, reducer=ee.Reducer.first(), scale=30)

            data = res.getInfo()['features']
            for f in data: results.append(f['properties'])
            print(f"   Batch {i} success")
        except Exception as e:
            print(f"   Batch {i} failed: {e}")
    return pd.DataFrame(results)

# Run Extraction
df_topo = extract_batch(stack_topo, feature_list, "Topography", 500)
df_s1 = extract_batch(stack_s1, feature_list, "Sentinel-1", 100)
df_s2 = extract_batch(stack_s2, feature_list, "Sentinel-2", 50)

--- Extracting Topography ---
   Batch 0 success
--- Extracting Sentinel-1 ---
   Batch 0 success
   Batch 100 success
--- Extracting Sentinel-2 ---
   Batch 0 success
   Batch 50 success
   Batch 100 success
   Batch 150 success


In [ ]:
# ============================================================================
# 5. MERGE
# ============================================================================
def merge_df(main, sub):
    if sub.empty: return main
    cols = ['core_id'] + [c for c in sub.columns if c not in ['core_id', 'system:index']]
    return pd.merge(main, sub[cols], on='core_id', how='left')

final = df_subset.copy()
final['core_id'] = final['core_id'].astype(str)
final = merge_df(final, df_topo)
final = merge_df(final, df_s1)
final = merge_df(final, df_s2)

final.to_csv('global_cores_with_gee_covariates.csv', index=False)
files.download('global_cores_with_gee_covariates.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>